<a href="https://colab.research.google.com/github/anupj/PyTorchForDeepLearningBootcamp/blob/main/07_pytorch_experiment_tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 07 PyTorch Experiment Tracking

Experiment Tracking allows us to determine which of our ML models works best, and therefore should be pursued.

Resources:
* https://www.learnpytorch.io/07_pytorch_experiment_tracking/
* https://madewithml.com/


## 0. Getting Setup

In [2]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.5.1+cu121
0.20.1+cu121


In [7]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

In [4]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [5]:
# Helper function to set seeds
def set_seeds(seed:int = 42):
  """Sets random seeds for torch operations.

  Args:
    seed(int) : the random seed that you'd like to set, defaults to 42"""

  # Set the general seed for torch operations
  torch.manual_seed(seed)

  # Set the seed for torch operations on the GPU
  torch.cuda.manual_seed(seed)


In [6]:
set_seeds()

## 01. Get Data

Get the sushi, steak, and pizza images

In [11]:
import os
import zipfile
from pathlib import Path
import requests

def download_data(source: str,
                  destination: str,
                  remove_source: bool = True) -> Path:
  """Downloads a zipped dataset from source and unzips to destination folder.
  Re-downloads if the destination directory is empty.
  """

  # Setup path to data folder
  data_path = Path("data/")
  image_path = data_path / destination

  # Check if the image folder exists and is not empty
  if image_path.is_dir() and any(image_path.iterdir()):
    print(f"[INFO] {image_path} directory already exists and is not empty, skipping download.")
  else:
    print(f"[INFO] {image_path} directory is empty or does not exist; Downloading it..")
    # Create the directory if it doesn't exist
    image_path.mkdir(parents=True, exist_ok=True)

    # Download and unzip the data (same as before)
    target_file = Path(source).name
    full_path = data_path / target_file

    with open(full_path, "wb") as f:
      request = requests.get(source)
      print(f"[INFO] Downloading {target_file} from {source}...")
      f.write(request.content)

    with zipfile.ZipFile(full_path, "r") as zip_ref:
      print(f"[INFO] Unzipping {target_file} data...")
      zip_ref.extractall(image_path)

    if remove_source:
      os.remove(full_path)
  return image_path

In [12]:
download_data(source = "https://github.com/mrdbourke/pytorch-deep-learning/raw/refs/heads/main/data/pizza_steak_sushi.zip", destination = "pizza_steak_sushi")

[INFO] data/pizza_steak_sushi directory is empty or does not exist; Downloading it..
[INFO] Downloading pizza_steak_sushi.zip from https://github.com/mrdbourke/pytorch-deep-learning/raw/refs/heads/main/data/pizza_steak_sushi.zip...
[INFO] Unzipping pizza_steak_sushi.zip data...


PosixPath('data/pizza_steak_sushi')